# 充足可能性問題と重み付き制約充足問題

## 重み付き制約充足問題

- 変数 $x_j \in D_j$ ($j = 1, \dots, n$) (各 $D_j$ は有限集合)
- 制約 $C_i \subset D_1 \times \dots \times D_n$ ($i = 1, \dots, m$)

制約充足問題とは, 上記条件を満たす $(x_1, \dots, x_n)$ を 1 組見つける(か解がないことを証明するか)問題のこと. 

制約からの逸脱量 $g_i(x)$ が定義できればこれを最小にする問題としても定式化できる. 
各制約の重みを $w_i$ として重み付き制約充足問題は

\begin{align}
&\text{minimize} \quad \sum_{i=1}^m w_i g_i(x) \\
&\text{s.t.} \quad x_j \in D_j \quad \text{(for $j = 1, \dots, n$)}
\end{align}

最適化ソルバー SCOP が紹介されているが, 時間があればやる...

## 時間割作成問題

### パラメータ

- 授業 $i$
- 教室 $k$
- 教員 $l$
- 学生
- 期 $t$: ここでは 1 週間(5 日もしくは 6 日）の各時限を考える. 

### 時間割を組む条件

- すべての授業をいずれか期へ割当
- すべての授業をいずれか教室へ割当
- 各期, 1 つの教室では 1 つ以下の授業
- 同じ教員の受持ち講義は異なる期へ
- 割当教室は受講学生数以上の容量をもつ
- 同じ学生が受ける可能性がある授業の集合(カリキュラム)は, 異なる期へ割り当てなければならない

### 付加条件

- 1 日の最後の期に割り当てられると履修学生数分のペナルティ
- 1 人の学生が履修する授業が 3 連続するとペナルティ
- 1 人の学生の授業数が 1 日に 1 つならばペナルティ(0 か 2 以上が望ましい)

### 定式化

#### 変数

- $x_{it}$: 授業 $i$ を期 $t$ に割り当てるとき $1$, そうでないとき $0$
- $y_{ik}$: 授業 $i$ を教室 $k$ に割り当てるとき $1$, そうでないとき $0$

#### 制約

- すべての授業をいずれか期へ割当
  - $\sum_{t} x_{it} = 1$ (for all $i$)
- すべての授業をいずれか教室へ割当
  - $\sum_{k} y_{ik} = 1$ (for all $i$)
- 各期 $t$ の各教室 $k$ への割り当て授業は $1$ 以下
  - $\sum_{i} x_{it} y_{ik} \leq 1$ (for all $t, k$) ※ AND 制約なので線形制約で表現可能
- 同じ教員の受持ち講義は異なる期へ割り当て
  - 教員 $l$ の受け持ち講義の集合を $E_l$ とする
  - $\sum_{i \in E_l} x_{it} \leq 1$ (for all $l, t$)
- 割当教室は受講学生数以上の容量をもつ
  - 授業 $i$ ができない(容量を超過する)教室の集合を $K_i$ とする
  - $\sum_i \sum_{k \in K_i} y_{ik} \leq 0$
- 同じ学生が受ける可能性がある授業の集合(カリキュラム)は異なる期へ割り当てなければならない
  - カリキュラム $j$ に含まれる授業の集合を $C_j$ とする
  - $\sum_{i \in C_j} x_{it} \leq 1$ (for all $j, t$)
- 1 日の最後の期に割り当てられると履修学生数分のペナルティ
  - 1日の最後の期の集合を $L$, 授業 $i$ の履修学生数を $w_i$ とする
  - $\sum_{i} \sum_{t \in L} w_i x_{it} \leq 0$
- 1 人の学生が履修する授業が 3 連続すると 1 ペナルティ
  - $T$ を 1 日のうちで最後の 2 時間でない期の集合とする
  - $\sum_{i \in C_j} (x_{i, t} + x_{i, t+1} + x_{i, t+2}) \leq 2$ (for all $t \in T$)
- 1 人の学生の授業数が 1 日に 1 つならば 1 ペナルティ(0 か 2 以上が望ましい)
  - 各日 $d$ に含まれる期の集合を $T_d$ とする
  - 日 $d$ におけるカリキュラム $j$ に含まれる授業数が $0$ か $2$ 以上なのかを表す 0-1 変数 $z_{jd}$ とする
  - $\sum_{t \in T_d} \sum_{i \in C_j} x_{it} \leq |T_d| z_{jd}$ (for all $d, j$)
  - $\sum_{t \in T_d} \sum_{i \in C_j} x_{it} \geq 2 z_{jd}$ (for all $d, j$)

## OR-tools (cp-sat)

In [1]:
from ortools.sat.python import cp_model
import ortools

ortools.__version__

'9.11.4210'

### 簡単な例

In [2]:
model = cp_model.CpModel()
var_upper_bound = max(50, 45, 37)
x = model.new_int_var(0, var_upper_bound, "x")
y = model.new_int_var(0, var_upper_bound, "y")
z = model.new_int_var(0, var_upper_bound, "z")

model.add(2 * x + 7 * y + 3 * z <= 50)
model.add(3 * x - 5 * y + 7 * z <= 45)
model.add(5 * x + 2 * y - 6 * z <= 37)

model.maximize(2 * x + 2 * y + 3 * z)

solver = cp_model.CpSolver()
status = solver.solve(model)

if status == cp_model.OPTIMAL:
    print("Maximum of objective function: %i" % solver.ObjectiveValue())
    print()
    print("x value: ", solver.Value(x))
    print("y value: ", solver.Value(y))
    print("z value: ", solver.Value(z))

Maximum of objective function: 35

x value:  7
y value:  3
z value:  5


### 全ての解の列挙

In [3]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print("%s=%i" % (v, self.value(v)), end=" ")
        print()

    @property
    def solution_count(self):
        return self.__solution_count


# Creates the model.
model = cp_model.CpModel()

# Creates the variables.
num_vals = 3
x = model.new_int_var(0, num_vals - 1, "x")
y = model.new_int_var(0, num_vals - 1, "y")
z = model.new_int_var(0, num_vals - 1, "z")

# Create the constraints.
model.add(x != y)

# Create a solver and solve.
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([x, y, z])
solver.parameters.enumerate_all_solutions = True
status = solver.solve(model, solution_printer)

print("Status = %s" % solver.StatusName(status))
print(f"Number of solutions found: {solution_printer.solution_count}")

x=1 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=1 
x=1 y=0 z=1 
x=2 y=1 z=1 
x=2 y=1 z=0 
x=2 y=1 z=2 
x=2 y=0 z=2 
x=1 y=0 z=2 
x=0 y=1 z=2 
x=0 y=1 z=1 
x=0 y=2 z=1 
x=0 y=2 z=2 
x=1 y=2 z=2 
x=1 y=2 z=1 
x=1 y=2 z=0 
x=0 y=2 z=0 
x=0 y=1 z=0 
Status = OPTIMAL
Number of solutions found: 18


### 例題: 覆面算

各文字に $0$ から $9$ の数字を入れて等式 $SEND+MORE=MONEY$ を成立させたい. 
ただし, 数字に重複があってはならず先頭の文字に $0$ を入れることはできない. 

In [4]:
model = cp_model.CpModel()
S = model.new_int_var(1,9,"S")
E = model.new_int_var(0,9,"E")
N = model.new_int_var(0,9,"N")
D = model.new_int_var(0,9,"D")
M = model.new_int_var(1,9,"M")
O = model.new_int_var(0,9,"O")
R = model.new_int_var(0,9,"R")
Y = model.new_int_var(0,9,"Y")

model.add(   1000*S + 100*E + 10*N + D + 
             1000*M + 100*O + 10*R + E 
 == 10000*M +1000*O + 100*N + 10*E + Y)

model.add_all_different([S,E,N,D,M,O,R,Y])

solver = cp_model.CpSolver()
status = solver.solve(model)

if status == cp_model.OPTIMAL:
    for v in [S,E,N,D,M,O,R,Y]:
        print(f"{v} = {solver.Value(v)}")

S = 9
E = 5
N = 6
D = 7
M = 1
O = 0
R = 8
Y = 2


In [5]:
solution_printer = VarArraySolutionPrinter([S,E,N,D,M,O,R,Y])
solver.parameters.enumerate_all_solutions = True
status = solver.solve(model, solution_printer)

print("Status = %s" % solver.StatusName(status))
print("Number of solutions found: %i" % solution_printer.solution_count)

S=9 E=5 N=6 D=7 M=1 O=0 R=8 Y=2 
Status = OPTIMAL
Number of solutions found: 1


### 練習問題: 覆面算

あとでやろう...

### 例題: 魔方陣

魔方陣とは $n \times n$の正方形の方陣に $1$ から $n^2$ までの整数を1つずつ入れて縦・横・対角線のいずれの列の和も同じになるものをいう. 
(一列の和は $n (n^2 + 1) / 2$ となる)

In [6]:
n = 13 # n = 12 より大きい値で試さないこと
model = cp_model.CpModel()
x = {}
for i in range(n):
    for j in range(n):
        x[i, j] = model.new_int_var(1, n * n, f"x({i},{j})")

x_list = [x[i,j] for i in range(n) for j in range(n)]
model.add_all_different(x_list)

s = n*(n**2+1)//2

for i in range(n):
    model.add(sum([x[i, j] for j in range(n)]) == s) 
for j in range(n):
    model.add(sum([x[i, j] for i in range(n)]) == s) 

model.add(sum([x[i, i] for i in range(n)]) == s) 
model.add(sum([x[i, n - i - 1] for i in range(n)]) == s)

solver = cp_model.CpSolver()
status = solver.solve(model)

for i in range(n):
    for j in range(n):
        print(solver.value(x[i,j]), end=" ")
    print()

90 1 44 12 34 37 143 160 42 94 158 126 164 
64 33 6 168 45 78 156 67 63 169 76 149 31 
127 167 120 51 11 84 95 135 134 22 49 3 107 
74 21 9 39 108 152 119 43 18 150 157 122 93 
30 154 75 155 140 70 142 131 72 15 52 29 40 
86 139 166 138 73 124 23 54 27 115 28 100 32 
114 16 57 41 103 125 83 5 121 77 117 87 159 
99 162 146 8 62 7 48 17 145 105 118 82 106 
137 136 163 69 153 65 25 130 61 36 13 98 19 
46 133 88 66 113 147 56 14 111 58 59 91 123 
151 68 101 161 38 4 79 148 50 47 97 81 80 
85 55 26 53 165 116 112 92 129 89 71 102 10 
2 20 104 144 60 96 24 109 132 128 110 35 141 


### 練習問題: 完全方陣

あとでやろう...

### 例題: 数独

数独は $9 \times 9$ の正方形の枠内に $1$ から $n$ までの数字を入れるパズルである. 
初期配置に与えられた数字はそのままとし, 縦・横の各列とブロックとよばれる $3 \times 3$ の小正方形の枠内には同じ数字を重複して入れてはいけないものとする. 

以下のデータでは, 数字の $0$ が入っている枠は空白であるとする. 

In [7]:
import math

problem = [
[1,0,0, 0,0,7, 0,9,0],
[0,3,0, 0,2,0, 0,0,8],
[0,0,9, 6,0,0, 5,0,0],

[0,0,5, 3,0,0, 9,0,0],
[0,1,0, 0,8,0, 0,0,2],
[6,0,0, 0,0,4, 0,0,0],

[3,0,0, 0,0,0, 0,1,0],
[0,4,0, 0,0,0, 0,0,7],
[0,0,7, 0,0,0, 3,0,0]]

model = cp_model.CpModel()  
n = len(problem)

cell_size = math.ceil(math.sqrt(n))
line_size = cell_size ** 2
line = range(0, line_size)
cell = range(0, cell_size)

x = {}
for i in line:
    for j in line:
        x[i, j] = model.new_int_var(1, line_size, f"x({i},{j})")

for i in line:
    model.add_all_different([x[i, j] for j in line])

for j in line:
    model.add_all_different([x[i, j] for i in line])

for i in cell:
    for j in cell:
        one_cell = []
        for di in cell:
            for dj in cell:
                one_cell.append(x[(i * cell_size + di, j * cell_size + dj)])
        model.add_all_different(one_cell)
    
for i in line:
    for j in line:
        if problem[i][j]:
            model.add(x[i, j] == problem[i][j])
            
solver = cp_model.CpSolver()
status = solver.solve(model)

for i in line:
    for j in line:
        print(solver.value(x[i,j]), end=" ")
    print()

1 6 2 8 5 7 4 9 3 
5 3 4 1 2 9 6 7 8 
7 8 9 6 4 3 5 2 1 
4 7 5 3 1 2 9 8 6 
9 1 3 5 8 6 7 4 2 
6 2 8 7 9 4 1 3 5 
3 5 6 4 7 8 2 1 9 
2 4 1 9 3 5 8 6 7 
8 9 7 2 6 1 3 5 4 


In [8]:
solution_printer = VarArraySolutionPrinter([x[i,j] for i in line for j in line])
solver.parameters.enumerate_all_solutions = True
status = solver.solve(model, solution_printer)

print("Status = %s" % solver.StatusName(status))
print("Number of solutions found: %i" % solution_printer.solution_count)

x(0,0)=1 x(0,1)=6 x(0,2)=2 x(0,3)=8 x(0,4)=5 x(0,5)=7 x(0,6)=4 x(0,7)=9 x(0,8)=3 x(1,0)=5 x(1,1)=3 x(1,2)=4 x(1,3)=1 x(1,4)=2 x(1,5)=9 x(1,6)=6 x(1,7)=7 x(1,8)=8 x(2,0)=7 x(2,1)=8 x(2,2)=9 x(2,3)=6 x(2,4)=4 x(2,5)=3 x(2,6)=5 x(2,7)=2 x(2,8)=1 x(3,0)=4 x(3,1)=7 x(3,2)=5 x(3,3)=3 x(3,4)=1 x(3,5)=2 x(3,6)=9 x(3,7)=8 x(3,8)=6 x(4,0)=9 x(4,1)=1 x(4,2)=3 x(4,3)=5 x(4,4)=8 x(4,5)=6 x(4,6)=7 x(4,7)=4 x(4,8)=2 x(5,0)=6 x(5,1)=2 x(5,2)=8 x(5,3)=7 x(5,4)=9 x(5,5)=4 x(5,6)=1 x(5,7)=3 x(5,8)=5 x(6,0)=3 x(6,1)=5 x(6,2)=6 x(6,3)=4 x(6,4)=7 x(6,5)=8 x(6,6)=2 x(6,7)=1 x(6,8)=9 x(7,0)=2 x(7,1)=4 x(7,2)=1 x(7,3)=9 x(7,4)=3 x(7,5)=5 x(7,6)=8 x(7,7)=6 x(7,8)=7 x(8,0)=8 x(8,1)=9 x(8,2)=7 x(8,3)=2 x(8,4)=6 x(8,5)=1 x(8,6)=3 x(8,7)=5 x(8,8)=4 
Status = OPTIMAL
Number of solutions found: 1


### 練習問題: 数独

気が向いたらやる...

### 練習問題: 不等式パズル

やっておきたい